In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.models import Model

In [6]:
!wget https://lazyprogrammer.me/course_files/spam.csv

--2020-11-23 10:35:16--  https://lazyprogrammer.me/course_files/spam.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.31.81.48, 104.31.80.48, 172.67.213.166, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.31.81.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/csv]
Saving to: ‘spam.csv’

spam.csv            100%[===================>] 491.86K   684KB/s    in 0.7s    

2020-11-23 10:35:17 (684 KB/s) - ‘spam.csv’ saved [503663/503663]

